# collate_met_and_seasonal_data_fire_mask_and_not

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from glob import glob
import os

In [2]:
drive = "D"
met_date = "20230109"
direction = "near"

feature_date = "20230128"

# processing date
date = "20230201"

# define output directory
output_dir = r"{0}:\cdu\data\zonal_stats\output\{1}".format(drive, date)


# Regression or classification
type_ = "class"
# type_ = "reg"


# Do you wish to add climate_data
climate = "yes"
# climate = "no"

# choose from one or the other to select fire mask applied or notscars or not
# choose for fire mask applied

# type_basal_feat_dir = "no_fire_scar_zonal_basal"
# fsm = "fms_applied"

# # choose for fire mask NOT applied
type_basal_feat_dir = "with_no_fire_scar_zonal_basal"
fsm = "fms_NOT_applied"





In [3]:
def mk_dir_fn(dir_):
    """ Create a new directory if one does not already exist. """
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

        
def export_csv_fn(list_, dir_, file_name):
    
    """ Create and export path from directory and file name and exports csv with no dropping the index column. """
    
    df_final = pd.concat(list_, axis =0)    
    output_path = os.path.join(dir_, file_name)
    df_final.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)
    
    return df_final


# def glob_concat_axis1_fn(d, s, m):
#     """ Concatenate mosaic basal and feature data based on site index. """
#     df_list = []

#     for f in glob(os.path.join(d, s)):

#         df = pd.read_csv(f)
# #         df.set_index("uid", inplace = True) # change back to uid once met data has been corrected
#         df.set_index("site", inplace = True)
#         df_list.append(df)
    
#     df_list.append(m)
#     # concat data
#     df_con = pd.concat(df_list, axis = 1)
    
#     # Drop duplicate columns
#     df_bm_dd = df_con.T.drop_duplicates().T
    

#     return df_list, df_bm_dd


def glob_concat_axis1_fn(d, s, m, climate):
    """ Concatenate mosaic basal and feature data based on site index. """
    df_list = []
    
    
    for f in glob(os.path.join(d, s)):
#         print(f)
        _, file_ = os.path.split(f)
        f_split = file_.split("_")
        
        old_s = f_split[0] 
#         print(old_s)
        new_s = f_split[0] + "_" + f_split[1]
#         print(new_s)

        df = pd.read_csv(f)
        col_list = df.columns.tolist()
        
        s_list = []
        for i in col_list:

            s = i.replace(old_s, new_s)
            s_list.append(s)
            
        df.columns = s_list

        
        # df.set_index("uid", inplace = True) # change back to uid once met data has been corrected
        df.set_index("site", inplace = True)
        df_list.append(df)
    
    
    if climate == "yes":
        df_list.append(m)
    else:
        pass
    
    # concat data
    df_con = pd.concat(df_list, axis = 1)
    
    # Drop duplicate columns
    df_bm_dd = df_con.T.drop_duplicates().T
    
    print(s_list)
    return df_list, df_bm_dd


def identify_null_values(df_in):
    """ Create and export column names and null values as a new df. """
    nc = df_in.isnull().sum()
    df_out = pd.DataFrame(nc[nc > 0].sort_values(ascending=False))
    for index, row in df_out.iterrows():
        print(f"{index}: {row[0]} Null values")

    df_out.reset_index(drop=False, inplace=True)
    
    return df_out


def drop_cols_fn(df, var1, var2, var3, var4, var5):
    """ Select out feature or stats values and export df's """
    df1 = df.copy(deep=True)
    df_columns = df1.columns.tolist()
    
    drop_list = [columns for columns in df_columns if var1 not in columns]  
    drop_list2 = [columns for columns in drop_list if var2 not in columns]
    drop_list3 = [columns for columns in drop_list2 if var3 not in columns]
    drop_list4 = [columns for columns in drop_list3 if var4 not in columns]
    drop_list5 = [columns for columns in drop_list4 if var5 not in columns]

    df1.drop(drop_list5, axis = 1, inplace=True)
    var_ = df.iloc[:, [0, 1, 11]]
    df_out = pd.concat([var_, df1], axis=1)
    

    return df1, var_, df_out

##### Create output directories

In [4]:
ml_data_dir = os.path.join(output_dir, "ml_data_dir")
mk_dir_fn(output_dir)
mk_dir_fn(ml_data_dir)

##### Load climate data 

In [5]:
m = r"{0}:\cdu\data\zonal_stats\output\{1}\cleaned_{2}_met_zonal_stats.csv".format(drive, met_date, direction)
print(m)

D:\cdu\data\zonal_stats\output\20230109\cleaned_near_met_zonal_stats.csv


In [6]:
m_df = pd.read_csv(m)
# set index to site or uid
m_df.set_index("site", inplace = True)
# m_df

In [7]:
# print(list(m_df))

##### Load basal feature zonal stats

In [8]:
bf = r"{0}:\cdu\data\zonal_stats\output\{1}\{2}".format(drive, feature_date, type_basal_feat_dir)
print(bf)

D:\cdu\data\zonal_stats\output\20230128\with_no_fire_scar_zonal_basal


In [9]:
# # Call the glob_concat_axis1 function to concatenate features
# bfm_df_list,df_bm_dd = glob_concat_axis1_fn(bf, "*.csv", m_df)

if climate == "yes":
    cl_var = "climate"
    # Call the glob_concat_axis1 function to concatenate features
    bfm_df_list,df_bm_dd = glob_concat_axis1_fn(bf, "*.csv", m_df, climate)
    
elif climate == "no":
    cl_var = "NO_climate"
    # Call the glob_concat_axis1 function to concatenate features
    bfm_df_list,df_bm_dd = glob_concat_axis1_fn(bf, "*.csv", m_df, climate)
    
else:
    print("ERROR")
    cl_var = "ERROR"
        


# call the drop columns function to select statistical values
taf_df, t_d_df, df_bm_dmmmms = drop_cols_fn(df_bm_dd, "mean", "max", "min", "med", "std")


['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'geometry', 'basal_dt', 'uid_dim_1202_1202', 'image', 's_day', 's_month', 's_year', 's_date', 'e_day', 'e_month', 'e_year', 'e_date', 'b1_dim_1202_count', 'b1_dim_1202_min', 'b1_dim_1202_max', 'b1_dim_1202_mean', 'b1_dim_1202_med', 'b1_dim_1202_std', 'b1_dim_1202_p25', 'b1_dim_1202_p50', 'b1_dim_1202_p75', 'b1_dim_1202_p95', 'b1_dim_1202_p99', 'b1_dim_1202_range', 'b2_dim_1202_count', 'b2_dim_1202_min', 'b2_dim_1202_max', 'b2_dim_1202_mean', 'b2_dim_1202_med', 'b2_dim_1202_std', 'b2_dim_1202_p25', 'b2_dim_1202_p50', 'b2_dim_1202_p75', 'b2_dim_1202_p95', 'b2_dim_1202_p99', 'b2_dim_1202_range', 'b3_dim_1202_count', 'b3_dim_1202_min', 'b3_dim_1202_max', 'b3_dim_1202_mean', 'b3_dim_1202_med', 'b3_dim_1202_std', 'b3

In [10]:
def drop_feature_fn(df):
    """ Remove features from dataframe """
    df1 = df.copy(deep=True)
    df_columns = df.columns.tolist()
    
    drop_list = [columns for columns in df_columns if 'dis_' in columns]
    drop_list2 = [columns for columns in df_columns if 'stc_' in columns]
    drop_list3 = [columns for columns in df_columns if 'mean.1' in columns]
    
    drop_list.extend(drop_list2)
    drop_list.extend(drop_list3)
    df1.drop(drop_list, axis = 1, inplace=True)
    
        
    return df1
 
    
dmmmms_nodisstc = drop_feature_fn(df_bm_dmmmms)

In [11]:
# dmmmms_nodisstc

In [12]:
def fill_min_list(df):
    """ Fill minimum values with 0 """
    
    df1 = df.copy(deep=True)
    
    df_columns = df.columns.tolist()
    min_list = [columns for columns in df_columns if '_min' in columns]
    
    for i in min_list:
#         print(i)

        df1.dropna(subset=[i], inplace=True)
        mean_value = df1[i].mean()
        print(f"feature {i} mean = {mean_value}")
        df[i].fillna(mean_value, inplace=True)
        
        
        #df[i].fillna((df1[i].mean()), inplace=True) #should be 0
         
    return df
 
    
dmmmms_min = fill_min_list(dmmmms_nodisstc)

feature b1_fpca2_0509_min mean = 6.803360925092326
feature b1_h99a_01122_min mean = 7.438527840335353
feature b1_dbi_0608_min mean = 432.91156462585036
feature b2_dbi_0608_min mean = 684.1088435374149
feature b3_dbi_0608_min mean = 974.0612244897959
feature b4_dbi_0608_min mean = 2121.3401360544217
feature b5_dbi_0608_min mean = 2393.6598639455783
feature b6_dbi_0608_min mean = 1481.6666666666667
feature b1_dja_0305_min mean = 13.863013698630137
feature b1_dp1_0112_min mean = 12.158620689655173
feature b2_dp1_0112_min mean = 22.03076923076923
feature b3_dp1_0112_min mean = 49.284615384615385
feature b1_dp1_0509_min mean = 11.733333333333333
feature b2_dp1_0509_min mean = 20.858407079646017
feature b3_dp1_0509_min mean = 51.16814159292036
feature b1_dim_0305_min mean = 6.884955752212389
feature b2_dim_0305_min mean = 37.690265486725664
feature b3_dim_0305_min mean = 38.10619469026549
feature b1_dim_0608_min mean = 12.955752212389381
feature b2_dim_0608_min mean = 21.98230088495575
featu

In [13]:
# df1 = dmmmms_nodisstc.copy()
# df2 = dmmmms_nodisstc

# df1.dropna(subset=["b1_dja_min"], inplace=True)
# mean_value = df1["b1_dja_min"].mean()
# # print(f"feature {"b1_dja_min"} mean = {mean_value}")
# #dmmmms_nodisstc["b1_dja_min"]= dmmmms_nodisstc["b1_dja_min"].fillna(mean_value, inplace=True)
# df2["b1_dja_min"].fillna(value=mean_value, inplace=True)

  


In [14]:
# print(dmmmms_nodisstc.shape)
# print(df2.shape)

In [15]:
# dmmmms_nodisstc.dropna(inplace=True)
# dmmmms_nodisstc.shape

In [16]:
# df2.dropna(inplace=True)
# df2.shape

In [17]:
# call the identify null values function
df_bm_dmmmms_null = identify_null_values(dmmmms_min)

b3_dp1_0509_std: 22 Null values
b1_dp1_0509_mean: 22 Null values
b3_dp1_0509_max: 22 Null values
b2_dp1_0509_med: 22 Null values
b2_dp1_0509_std: 22 Null values
b2_dp1_0509_mean: 22 Null values
b2_dp1_0509_max: 22 Null values
b1_dp1_0509_med: 22 Null values
b1_dp1_0509_std: 22 Null values
b1_dp1_0509_max: 22 Null values
b3_dp1_0509_med: 22 Null values
b3_dp1_0509_mean: 22 Null values
b1_h99a_01122_std: 20 Null values
b1_h99a_01122_med: 20 Null values
b1_h99a_01122_mean: 20 Null values
b1_h99a_01122_max: 20 Null values
b1_fpca2_0509_std: 13 Null values
b1_fpca2_0509_med: 13 Null values
b1_fpca2_0509_mean: 13 Null values
b1_fpca2_0509_max: 13 Null values
b1_dp1_0112_std: 3 Null values
b1_dp1_0112_med: 3 Null values
b3_dp1_0112_std: 3 Null values
b3_dp1_0112_med: 3 Null values
b3_dp1_0112_mean: 3 Null values
b3_dp1_0112_max: 3 Null values
b2_dp1_0112_med: 3 Null values
b2_dp1_0112_std: 3 Null values
b2_dp1_0112_mean: 3 Null values
b1_dp1_0112_max: 3 Null values
b1_dp1_0112_mean: 3 Null va

In [18]:
def mean_null(df):
    """ Mean the null values of data excluding date and biomass."""
    df1 = df.copy(deep=True)

    df_columns = df1.columns.tolist()

    for i in df_columns[3:]:
        df1.dropna(subset=[i], inplace=True)
        mean_value = df1[i].mean()
        print(f"feature {i} mean = {mean_value}")
        df[i].fillna(mean_value, inplace=True)

        
    return df
    

In [19]:
dmmmms_mean = mean_null(dmmmms_min)

feature b1_fpca2_0509_min mean = 6.803360925092326
feature b1_fpca2_0509_max mean = 17.199115225246974
feature b1_fpca2_0509_mean mean = 11.24807630166116
feature b1_fpca2_0509_med mean = 11.001351992805283
feature b1_fpca2_0509_std mean = 2.7999219686105583
feature b1_h99a_01122_min mean = 7.438527840335353
feature b1_h99a_01122_max mean = 12.225477257553411
feature b1_h99a_01122_mean mean = 9.74961009156875
feature b1_h99a_01122_med mean = 9.727706052819077
feature b1_h99a_01122_std mean = 1.278184918888929
feature b1_dbi_0608_min mean = 432.91156462585036
feature b1_dbi_0608_max mean = 561.7210884353741
feature b1_dbi_0608_mean mean = 493.24863945578244
feature b1_dbi_0608_med mean = 491.90816326530614
feature b1_dbi_0608_std mean = 35.034468210311935
feature b2_dbi_0608_min mean = 684.1088435374149
feature b2_dbi_0608_max mean = 863.4013605442177
feature b2_dbi_0608_mean mean = 769.0147108843538
feature b2_dbi_0608_med mean = 766.7482993197278
feature b2_dbi_0608_std mean = 49.3809

In [20]:
dmmmms_mean

,uid,date,bio_agb_kg1ha,b1_fpca2_0509_min,b1_fpca2_0509_max,b1_fpca2_0509_mean,b1_fpca2_0509_med,b1_fpca2_0509_std,b1_h99a_01122_min,b1_h99a_01122_max,...,evp_s_mean,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean
site,,,,,,,,,,,,,,,,,,,,,
nt001,119,20110523,0,13.779837,46.998966,25.849381,23.867432,8.477655,7.273514,11.780785,...,5.2,32.6,16.3,141.6,1013.9,16.7,3311.1,3368.3,17.0,22.0
barkely01,42,20111025,0,0.752944,2.872306,1.691590,1.486260,0.699975,4.384300,8.969109,...,8.7,37.0,23.6,71.7,1012.3,16.9,3312.2,3353.4,22.4,29.8
barkely02,43,20111026,0,2.120632,13.779837,4.974984,4.725986,2.433672,3.129379,5.818588,...,7.9,38.6,23.9,71.5,1012.1,19.2,3311.7,3357.8,24.1,31.9
barkely04,45,20111026,0,2.120632,6.417120,4.374468,4.472551,1.216005,5.932899,8.854515,...,7.8,38.6,23.9,72.9,1012.1,19.2,3311.9,3358.1,24.2,31.8
barkely03,44,20111026,0,0.752944,3.292108,1.820417,1.637530,0.699339,3.664345,7.094025,...,7.8,38.5,23.8,73.3,1012.1,19.2,3312.2,3359.0,24.3,31.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ntadac0002,5,20160506,20805.2,15.538734,42.389004,29.117230,29.265415,6.836081,16.053879,20.420395,...,5.7,35.1,24.6,228.9,1011.2,18.5,3326.9,3368.7,28.5,20.3
ntaarp0001,1,20160602,30472.4,27.614885,38.921173,34.748572,34.327137,3.238761,17.312332,21.588921,...,6.0,33.1,20.7,203.8,1014.4,19.6,3323.8,3374.4,23.9,18.5
ntaarp0002,2,20160602,24414.1,9.056872,22.308666,14.395230,14.214185,3.600052,9.738800,16.808601,...,6.0,32.9,20.7,204.2,1014.5,19.6,3324.1,3374.0,23.8,18.3


In [21]:
# call the identify null values function
df_bm_dmmmms_null = identify_null_values(dmmmms_mean)

In [22]:
dmmmms_mean.dropna(inplace=True)

In [23]:
dmmmms_mean

,uid,date,bio_agb_kg1ha,b1_fpca2_0509_min,b1_fpca2_0509_max,b1_fpca2_0509_mean,b1_fpca2_0509_med,b1_fpca2_0509_std,b1_h99a_01122_min,b1_h99a_01122_max,...,evp_s_mean,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean
site,,,,,,,,,,,,,,,,,,,,,
nt001,119,20110523,0,13.779837,46.998966,25.849381,23.867432,8.477655,7.273514,11.780785,...,5.2,32.6,16.3,141.6,1013.9,16.7,3311.1,3368.3,17.0,22.0
barkely01,42,20111025,0,0.752944,2.872306,1.691590,1.486260,0.699975,4.384300,8.969109,...,8.7,37.0,23.6,71.7,1012.3,16.9,3312.2,3353.4,22.4,29.8
barkely02,43,20111026,0,2.120632,13.779837,4.974984,4.725986,2.433672,3.129379,5.818588,...,7.9,38.6,23.9,71.5,1012.1,19.2,3311.7,3357.8,24.1,31.9
barkely04,45,20111026,0,2.120632,6.417120,4.374468,4.472551,1.216005,5.932899,8.854515,...,7.8,38.6,23.9,72.9,1012.1,19.2,3311.9,3358.1,24.2,31.8
barkely03,44,20111026,0,0.752944,3.292108,1.820417,1.637530,0.699339,3.664345,7.094025,...,7.8,38.5,23.8,73.3,1012.1,19.2,3312.2,3359.0,24.3,31.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ntadac0002,5,20160506,20805.2,15.538734,42.389004,29.117230,29.265415,6.836081,16.053879,20.420395,...,5.7,35.1,24.6,228.9,1011.2,18.5,3326.9,3368.7,28.5,20.3
ntaarp0001,1,20160602,30472.4,27.614885,38.921173,34.748572,34.327137,3.238761,17.312332,21.588921,...,6.0,33.1,20.7,203.8,1014.4,19.6,3323.8,3374.4,23.9,18.5
ntaarp0002,2,20160602,24414.1,9.056872,22.308666,14.395230,14.214185,3.600052,9.738800,16.808601,...,6.0,32.9,20.7,204.2,1014.5,19.6,3324.1,3374.0,23.8,18.3


In [24]:
dmmmms_mean.reset_index(drop=False, inplace=True)
dmmmms_mean.rename(columns={"index": "site"}, inplace=True)
dmmmms_mean.dropna(inplace=True)

In [25]:
# dmmmms_mean.to_csv(os.path.join(ml_data_dir, f"season_met_fillna_mean_{fsm}_applied.csv"))

In [26]:
path_ = os.path.join(ml_data_dir, f"season_{cl_var}_fillna_mean_{fsm}_{type_}.csv")      
dmmmms_mean.to_csv(path_)
print(path_)

D:\cdu\data\zonal_stats\output\20230201\ml_data_dir\season_climate_fillna_mean_fms_NOT_applied_class.csv


In [27]:
def drop_low_r2_seasons_fn(df):
    """ Remove features from dataframe """
    df_columns = df.columns.tolist()
    
    drop_list = [columns for columns in df_columns if '0305' in columns]
    drop_list2 = [columns for columns in df_columns if '1202' in columns]
    drop_list3 = [columns for columns in df_columns if 'dp1_0509' in columns]
    
    drop_list.extend(drop_list2)
    drop_list.extend(drop_list3)
    df.drop(drop_list, axis = 1, inplace=True)
    
        
    return df
 
    
selected_features_and_seasons = drop_feature_fn(dmmmms_mean)

In [28]:
# path_ = os.path.join(ml_data_dir, f"r2_best_season_{cl_var}_fillna_{fsm}_{type_}.csv")      
# selected_features_and_seasons.to_csv(path_)
# print(path_)

In [29]:
def classifier(row):
    if row["bio_agb_kg1ha"] == 0:
        return "c0"
    elif 0 > row["bio_agb_kg1ha"] < 10000:
        return "c1"
    elif 10000 > row["bio_agb_kg1ha"] < 20000:
        return "c2"
    elif 20000 > row["bio_agb_kg1ha"] < 30000:
        return "c3"
    elif 30000 > row["bio_agb_kg1ha"] < 40000:
        return "c4"
    elif 40000 > row["bio_agb_kg1ha"] < 50000:
        return "c5"
    else:
        return "c6"

In [30]:
if type_ == "class":
    
    dmmmms_mean["target"] = dmmmms_mean.apply(classifier, axis=1)
#     dmmmms_mean.reset_index(drop=False)
#     dmmmms_mean.rename({"index": "site"}, inplace=True)
#     dmmmms_mean.dropna(inplace=True)

    path_ = os.path.join(ml_data_dir, f"season_{cl_var}_fillna_mean_{fsm}_{type_}.csv")      
    dmmmms_mean.to_csv(path_, index = False)
    print(path_)

    selected_features_and_seasons["target"] = selected_features_and_seasons.apply(classifier, axis=1)
    
#     selected_features_and_seasons.reset_index(drop=False)
#     selected_features_and_seasons.rename({"index": "site"}, inplace=True)
#     selected_features_and_seasons.dropna(inplace=True)
    
    path_ = os.path.join(ml_data_dir, f"r2_best_season_{cl_var}_fillna_{fsm}_{type_}.csv")      
    selected_features_and_seasons.to_csv(path_, index = False)
    print(path_)

else:
    dmmmms_mean["target"] = dmmmms_mean["bio_agb_kg1ha"]
    
#     dmmmms_mean["target"] = dmmmms_mean.apply(classifier, axis=1)
#     dmmmms_mean.reset_index(drop=False)
#     dmmmms_mean.rename({"index": "site"}, inplace=True)
#     dmmmms_mean.dropna(inplace=True)
    
    path_ = os.path.join(ml_data_dir, f"season_{cl_var}_fillna_mean_{fsm}_{type_}.csv")      
    dmmmms_mean.to_csv(path_, index = False)
    print(path_)
    
    selected_features_and_seasons["target"] = selected_features_and_seasons["bio_agb_kg1ha"]
    
#     selected_features_and_seasons.reset_index(drop=False)
#     selected_features_and_seasons.rename({"index": "site"}, inplace=True)
#     selected_features_and_seasons.dropna(inplace=True)
    
    path_ = os.path.join(ml_data_dir, f"r2_best_season_{cl_var}_fillna_{fsm}_{type_}.csv")      
    selected_features_and_seasons.to_csv(path_, index = False)
    print(path_)    

D:\cdu\data\zonal_stats\output\20230201\ml_data_dir\season_climate_fillna_mean_fms_NOT_applied_class.csv
D:\cdu\data\zonal_stats\output\20230201\ml_data_dir\r2_best_season_climate_fillna_fms_NOT_applied_class.csv


In [31]:
selected_features_and_seasons

,site,uid,date,bio_agb_kg1ha,b1_fpca2_0509_min,b1_fpca2_0509_max,b1_fpca2_0509_mean,b1_fpca2_0509_med,b1_fpca2_0509_std,b1_h99a_01122_min,...,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean,target
0,nt001,119,20110523,0,13.779837,46.998966,25.849381,23.867432,8.477655,7.273514,...,32.6,16.3,141.6,1013.9,16.7,3311.1,3368.3,17.0,22.0,c0
1,barkely01,42,20111025,0,0.752944,2.872306,1.691590,1.486260,0.699975,4.384300,...,37.0,23.6,71.7,1012.3,16.9,3312.2,3353.4,22.4,29.8,c0
2,barkely02,43,20111026,0,2.120632,13.779837,4.974984,4.725986,2.433672,3.129379,...,38.6,23.9,71.5,1012.1,19.2,3311.7,3357.8,24.1,31.9,c0
3,barkely04,45,20111026,0,2.120632,6.417120,4.374468,4.472551,1.216005,5.932899,...,38.6,23.9,72.9,1012.1,19.2,3311.9,3358.1,24.2,31.8,c0
4,barkely03,44,20111026,0,0.752944,3.292108,1.820417,1.637530,0.699339,3.664345,...,38.5,23.8,73.3,1012.1,19.2,3312.2,3359.0,24.3,31.4,c0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,ntadac0002,5,20160506,20805.2,15.538734,42.389004,29.117230,29.265415,6.836081,16.053879,...,35.1,24.6,228.9,1011.2,18.5,3326.9,3368.7,28.5,20.3,c4
163,ntaarp0001,1,20160602,30472.4,27.614885,38.921173,34.748572,34.327137,3.238761,17.312332,...,33.1,20.7,203.8,1014.4,19.6,3323.8,3374.4,23.9,18.5,c5
164,ntaarp0002,2,20160602,24414.1,9.056872,22.308666,14.395230,14.214185,3.600052,9.738800,...,32.9,20.7,204.2,1014.5,19.6,3324.1,3374.0,23.8,18.3,c4
165,ntaarp0003,3,20160603,17598.4,5.261469,25.453907,13.082606,12.109756,5.789599,9.613670,...,35.1,21.1,196.1,1014.7,17.7,3318.4,3371.3,23.7,22.8,c3


In [32]:
dmmmms_mean

,site,uid,date,bio_agb_kg1ha,b1_fpca2_0509_min,b1_fpca2_0509_max,b1_fpca2_0509_mean,b1_fpca2_0509_med,b1_fpca2_0509_std,b1_h99a_01122_min,...,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean,target
0,nt001,119,20110523,0,13.779837,46.998966,25.849381,23.867432,8.477655,7.273514,...,32.6,16.3,141.6,1013.9,16.7,3311.1,3368.3,17.0,22.0,c0
1,barkely01,42,20111025,0,0.752944,2.872306,1.691590,1.486260,0.699975,4.384300,...,37.0,23.6,71.7,1012.3,16.9,3312.2,3353.4,22.4,29.8,c0
2,barkely02,43,20111026,0,2.120632,13.779837,4.974984,4.725986,2.433672,3.129379,...,38.6,23.9,71.5,1012.1,19.2,3311.7,3357.8,24.1,31.9,c0
3,barkely04,45,20111026,0,2.120632,6.417120,4.374468,4.472551,1.216005,5.932899,...,38.6,23.9,72.9,1012.1,19.2,3311.9,3358.1,24.2,31.8,c0
4,barkely03,44,20111026,0,0.752944,3.292108,1.820417,1.637530,0.699339,3.664345,...,38.5,23.8,73.3,1012.1,19.2,3312.2,3359.0,24.3,31.4,c0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,ntadac0002,5,20160506,20805.2,15.538734,42.389004,29.117230,29.265415,6.836081,16.053879,...,35.1,24.6,228.9,1011.2,18.5,3326.9,3368.7,28.5,20.3,c4
163,ntaarp0001,1,20160602,30472.4,27.614885,38.921173,34.748572,34.327137,3.238761,17.312332,...,33.1,20.7,203.8,1014.4,19.6,3323.8,3374.4,23.9,18.5,c5
164,ntaarp0002,2,20160602,24414.1,9.056872,22.308666,14.395230,14.214185,3.600052,9.738800,...,32.9,20.7,204.2,1014.5,19.6,3324.1,3374.0,23.8,18.3,c4
165,ntaarp0003,3,20160603,17598.4,5.261469,25.453907,13.082606,12.109756,5.789599,9.613670,...,35.1,21.1,196.1,1014.7,17.7,3318.4,3371.3,23.7,22.8,c3
